# EE 564 Project 2

## Tesla Model S Motor Design


[Tesla Model S](http://www.teslamotors.com/models)

[RWD 85 Model](http://en.wikipedia.org/wiki/Tesla_Model_S#Powertrain)

* Max. Power: 360 hp (270 kW)
* Max. Torque: 441 Nm
* Top Speed: 225 km/h 

[Specs1](http://teslatap.com/undocumented/) and [Specs2](http://my.teslamotors.com/it_IT/forum/forums/tesla-model-s-motor-output)

**Useful Links:**

* [How does Tesla make their motors in the model S so small?](http://www.reddit.com/r/AskEngineers/comments/2mv2h5/how_does_tesla_make_their_motors_in_the_model_s/)

* [Tesla Motor Technology](http://my.teslamotors.com/roadster/technology/motor)

* [Electric traction machine choice](http://sites.ieee.org/miami/files/2014/11/Hendershot-FIU-Lecture.pdf)

In [134]:
Pn = 270000
ntarget = 0.96
Vln=600
f1=60
p1=2
m=3

In [135]:
import math
KE=0.98-0.005*p1
cosphin=0.9
nn=0.96
Sgap=KE*Pn/(nn*cosphin)
print KE
print Sgap

0.97
303125.0


In [136]:
C0=250e3
lambda1=1.5
Dis=math.pow((2*p1*p1*Sgap)/(3.14*lambda1*f1*C0),1.0/3.0)
print Dis

0.324987401643


In [137]:
g=1.5e-3
tau=3.14*Dis/(2*p1)
L=lambda1*tau
print L

0.382672665434


In [138]:
q=3
taus=tau/(3*q)
print tau
print taus

0.255115110289
0.0283461233655


In [139]:
KD=0.62
Dout=Dis/KD
print Dout

0.524173228456


In [140]:
g=(0.1+0.012*math.pow(Pn,1.0/3.0))*1e-3
print g

0.000875596488411


# Stator Winding

In [141]:
Ns=2*p1*q*m
print Ns

36


In [142]:
alphaec=2*3.14*p1/Ns
print alphaec

0.348888888889


In [143]:
Kq1=math.sin(3.14/(2*m))/(q*math.sin(3.14/(2*m*q)))
print Kq1

0.959835420315


In [144]:
Ky1=math.sin(3.14/2*7/9)
print Ky1

0.939480605157


In [145]:
Kw1=Kq1*Ky1
print Kw1

0.901746761529


In [146]:
Bg=0.7
Kf=1.085
alphai=0.729
phi=alphai*tau*L*Bg
print phi


0.0498183330897


In [147]:
W1=KE*Vln/(4*Kf*Kw1*f1*phi)
print W1

49.751733951


In [148]:
a1=1
ns=a1*W1/(p1*q)
print ns

8.29195565851


In [149]:
ns=8
W1=p1*q*ns
print W1

48


In [150]:
Bg=0.7*49.75/48
print Bg

0.725520833333


In [151]:
Iln=Pn/(3*Vln*cosphin*nn)
print Iln

173.611111111


In [152]:
Jcos=5
Aco=Iln/(Jcos*a1)
print Aco

34.7222222222


In [153]:
dco=math.sqrt(4*Aco/3.14)
print dco

6.65072403723


In [154]:
A1=C0/(Kf*alphai*Kw1*3.14*Bg*2*p1)
print A1

38464.3700122
